In [59]:
from data.process_data import load_data, save_data
from models.train_classifier import (
    load_data,
    tokenize,
    build_model,
    evaluate_model,
    save_model,
)
import pandas as pd
from sqlalchemy import create_engine
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split


nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /Users/dominik/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dominik/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/dominik/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [35]:
df = load_data("./data/disaster_messages.csv", "./data/disaster_categories.csv")

In [44]:
# Copy dataframe to local dataframe
df_clean = df
# Split category into subcategories
categories = df.categories.str.split(";", expand=True)
# Label columns according to new label
categories.columns = categories.iloc[0].str[:-2]
# Make columns numeric, i.e. remove the label substring from the content
for label, content in categories.iteritems():
    categories[label] = pd.to_numeric(content.str.replace(f"{label}-", ""))
# Clean related category to 0/1 - there are outliers with 2s
categories["related"] = categories["related"].map(lambda x: 1 if x == 2 else x)
# Drop original category column    
df_clean = df_clean.drop(labels='categories', axis=1)
# Add categories to dataframe
df_clean = df_clean.join(categories)



In [45]:
df_clean.describe()

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26386.000000,26386.000000,26386.000000,26386.000000,26386.000000,26386.00000,26386.000000,26386.000000,26386.00000,26386.000000,...,26386.000000,26386.000000,26386.000000,26386.000000,26386.000000,26386.000000,26386.000000,26386.000000,26386.000000,26386.000000
mean,15217.885886,0.767301,0.171038,0.004586,0.415144,0.07955,0.049989,0.027477,0.01785,0.032707,...,0.011711,0.043773,0.278292,0.082506,0.093383,0.010687,0.093269,0.020200,0.052263,0.193777
std,8823.741128,0.422560,0.376549,0.067564,0.492756,0.27060,0.217926,0.163471,0.13241,0.177871,...,0.107583,0.204594,0.448166,0.275139,0.290974,0.102828,0.290815,0.140687,0.222560,0.395264
min,2.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7438.250000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15650.500000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,22916.750000,1.000000,0.000000,0.000000,1.000000,0.00000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,30265.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.00000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [37]:
categories

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26381,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26382,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26383,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26384,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
df_clean

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26381,30261,The training demonstrated how to enhance micro...,NaN,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26382,30262,A suitable candidate has been selected and OCH...,NaN,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26383,30263,"Proshika, operating in Cox's Bazar municipalit...",NaN,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26384,30264,"Some 2,000 women protesting against the conduc...",NaN,news,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
database_filepath = "data/DisasterResponse.db"
engine = create_engine(f'sqlite:///{database_filepath}')
df = pd.read_sql_table('YourTableName', con = engine)

X = df["message"]
Y = df.drop(["id", "message", "original", "genre"], axis=1)
category_names = Y.columns.to_list()

X, Y, category_names

(0        Weather update - a cold front from Cuba that c...
 1                  Is the Hurricane over or is it not over
 2                          Looking for someone but no name
 3        UN reports Leogane 80-90 destroyed. Only Hospi...
 4        says: west side of Haiti, rest of the country ...
                                ...                        
 26381    The training demonstrated how to enhance micro...
 26382    A suitable candidate has been selected and OCH...
 26383    Proshika, operating in Cox's Bazar municipalit...
 26384    Some 2,000 women protesting against the conduc...
 26385    A radical shift in thinking came about as a re...
 Name: message, Length: 26386, dtype: object,
        related  request  offer  aid_related  medical_help  medical_products  \
 0            1        0      0            0             0                 0   
 1            1        0      0            1             0                 0   
 2            1        0      0            0          

In [55]:
category_names.to_list()

['related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'child_alone',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

In [60]:
database_filepath = "data/DisasterResponse.db"
model_filepath = "models/classifier.pkl"
print("Loading data...\n    DATABASE: {}".format(database_filepath))
X, Y, category_names = load_data(database_filepath)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

print("Building model...")
model = build_model()

print("Training model...")
model.fit(X_train, Y_train)

print("Evaluating model...")
evaluate_model(model, X_test, Y_test, category_names)

print("Saving model...\n    MODEL: {}".format(model_filepath))
save_model(model, model_filepath)

print("Trained model saved!")

Loading data...
    DATABASE: data/DisasterResponse.db
Building model...
Training model...
Evaluating model...


ValueError: multilabel-indicator is not supported

In [91]:
from sklearn.metrics import classification_report

Y_pred = model.predict(X_test)

target_names = category_names
classification_report = classification_report(
    Y_test, Y_pred, target_names=target_names, zero_division=0
)
accuracy = (Y_pred == Y_test).mean()

print("Classification Report:\n", classification_report)
print("Accuracy:\n", accuracy)

Classification Report:
                         precision    recall  f1-score   support

               related       0.85      0.94      0.90      4069
               request       0.84      0.49      0.62       923
                 offer       1.00      0.07      0.14        27
           aid_related       0.81      0.53      0.65      2269
          medical_help       0.82      0.02      0.04       419
      medical_products       0.87      0.05      0.09       264
     search_and_rescue       0.44      0.03      0.05       157
              security       0.00      0.00      0.00        90
              military       0.47      0.05      0.10       148
           child_alone       0.00      0.00      0.00         0
                 water       0.90      0.27      0.42       328
                  food       0.85      0.47      0.60       595
               shelter       0.89      0.25      0.39       471
              clothing       0.67      0.12      0.21        82
               

In [66]:
print(len(category_names))
print(len(Y_pred))
print(len(Y_test))

36
5278
5278


In [105]:
df.iloc[:,4:].sum().sort_values(ascending=False).index.tolist()

['related',
 'aid_related',
 'weather_related',
 'direct_report',
 'request',
 'other_aid',
 'food',
 'storm',
 'earthquake',
 'shelter',
 'floods',
 'medical_help',
 'infrastructure_related',
 'water',
 'other_weather',
 'buildings',
 'medical_products',
 'transport',
 'death',
 'other_infrastructure',
 'refugees',
 'military',
 'search_and_rescue',
 'money',
 'electricity',
 'cold',
 'security',
 'clothing',
 'aid_centers',
 'missing_people',
 'hospitals',
 'fire',
 'tools',
 'offer',
 'shops',
 'child_alone']

In [132]:
top20_words_names

['water',
 'people',
 'food',
 'help',
 'need',
 'please',
 'earthquake',
 'u',
 'area',
 'like',
 'would',
 'said',
 '000',
 'http',
 'country',
 'know',
 'also',
 'government',
 'haiti',
 'one']